**HOMEWORK**

* **1bp** Implement a version of *Grid Search Cross Validation* to search for the best combination of hyperparameters and callbacks, optimizing the classification stage for the EFD-MNIST dataset. Report your results comparing the best three models.


In [1]:
! pip install pyefd
! pip install keras
! pip install tensorflow

You should consider upgrading via the 'C:\Users\GeorgeS\Documents\facultate\master\an1\s2\UAIC_FII_CV\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\GeorgeS\Documents\facultate\master\an1\s2\UAIC_FII_CV\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\GeorgeS\Documents\facultate\master\an1\s2\UAIC_FII_CV\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import tensorflow.keras.datasets as datasets
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.metrics import accuracy_score, confusion_matrix

(X_train, Y_train), (X_test, Y_test) = datasets.mnist.load_data()


In [2]:

import cv2 as cv
import numpy as np
import os
from pyefd import elliptic_fourier_descriptors



def load_dataset(dset_dir):
  # a dict to keep the class of each image
  image_index = {}

  for class_name in os.listdir(dset_dir):
    in_cat_images = []
    cat_dir_path = "{}/{}".format(dset_dir, class_name)
    for file_name in os.listdir(cat_dir_path):
      img_path = "{}/{}".format(cat_dir_path, file_name)
      img = cv.imread(img_path, 0) # open as grayscale
      if img is not None: # unless some corrupted images
        in_cat_images.append(img)
    image_index[class_name] = in_cat_images

  return image_index


def extract_efd_feature(img):
  contours, _ = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  fd = []
  for cnt in contours:
    coeffs = elliptic_fourier_descriptors(np.squeeze(cnt), order=10, normalize=True)
    fd.append(coeffs.flatten()[3:])
  return np.array(fd)


def prepare_data(image_set):
  X = []
  Y = []
  for img_class, img_list in image_set.items():
    for image in img_list:
      try:
        fdesc = extract_efd_feature(image)
        X.append(np.max(fdesc, axis=0))
        Y.append(int(img_class))
      except np.AxisError:
        continue # some corrupted images

  return np.array(X), np.array(Y)

In [3]:

train_img = load_dataset("mnist_png_demo/training/")
val_img = load_dataset("mnist_png_demo/validation/")
test_img = load_dataset("mnist_png_demo/testing/")

X_train, Y_train = prepare_data(train_img)
X_val, Y_val = prepare_data(val_img)
X_test, Y_test = prepare_data(test_img)

In [10]:

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

def create_model(layers, optimizer):
    model = Sequential()
    # print(layers)
    for sz, activation in layers:
        model.add(Dense(sz, activation=activation))
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model





In [11]:
optimizers = [
  "adam",
  "sgd"
]
layers = [
  [
    (64, "sigmoid"),
    (32, "sigmoid")
  ],
  [
    (64, "sigmoid"),
    (32, "relu")
  ],
  [
    (64, "relu"),
    (32, "sigmoid")
  ],
   [
    (64, "relu"),
    (32, "relu")
  ]
]

best = 0
best_layers = None
best_opt = None

for optimizer in optimizers:
  for layer in layers:
    model = create_model(layer, optimizer)
    model.fit(X_train, Y_train, epochs=60, verbose=0)
    predict_x = model.predict(X_test)
    predictions = np.argmax(predict_x, axis=1)
    rez = accuracy_score(Y_test, predictions)
    if rez > best:
      best = rez
      best_opt = optimizer
      best_layers = layer

print(best)
print(best_opt)
print(best_layers)

c:\users\georges\documents\facultate\master\an1\s2\uaic_fii_cv\pythonproject\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
c:\users\georges\documents\facultate\master\an1\s2\uaic_fii_cv\pythonproject\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
c:\users\georges\documents\facultate\master\an1\s2\uaic_fii_cv\pythonproject\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, bu

0.865
adam
[(64, 'relu'), (32, 'relu')]


* **1bp** Compare the Auto-Encoder learnt representation to the EFD fitted classifiers from *week 8*. Then re-fit your dataset removing the non linear activations (i. e. use activation a(x) = x). Is the autoencoder able to learn a better representation? Discuss your results.

When using activation a(x) = x, the autoencoder is not able to learn the representation
at all.
This happens due to the fact that there's no sparsity, and therefore it's hard to reduce
dimensionality with linear functions.


In [15]:

import keras
from keras import regularizers
from keras.layers import Dense, Dropout

# same as the EFD descriptor
latent_dim = 37

input_flat = keras.Input(shape=(784, )) # 28 x 28
encoded = Dense(128)(input_flat)
encoded = Dense(64)(encoded)
latent = Dense(latent_dim)(encoded)

decoded = Dense(64)(latent)
decoded = Dense(128)(decoded)
reconstruction = Dense(784)(decoded) # apply the logistic model

autoencoder = keras.Model(input_flat, reconstruction)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# mse loss between input and output


In [21]:

import numpy as np
from keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [7]:

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))



Epoch 1/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0377 - accuracy: 0.0100 - val_loss: 0.0207 - val_accuracy: 0.0107

* **2bp** Using the *Cross Validation* procedure, determine the best architecture format for the Multi-Layer Perceptron model, trained on the MNIST dataset. Use dropout to regularize your model. Estimate the number of parameters used in training/inference.


In [8]:
!pip install tpot

You should consider upgrading via the 'C:\Users\GeorgeS\Documents\facultate\master\an1\s2\UAIC_FII_CV\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:

def create_model(layers, optimizer):
  model = Sequential()
  model.add(Flatten())
  for size, act in layers:
    model.add(Dense(size, activation=act))
    model.add(Dropout(0.2))

  model.add(Dense(10)) # the definition of the classifier stage will be the same
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model


optimizers = [
  "adam",
  "sgd"
]
layers = [
  [
    (64, "sigmoid"),
    (32, "sigmoid")
  ],
  [
    (64, "sigmoid"),
    (32, "relu")
  ],
  [
    (64, "relu"),
    (32, "sigmoid")
  ],
   [
    (64, "relu"),
    (32, "relu")
  ]
]


In [22]:
best = 0
best_layers = None
best_opt = None

for optimizer in optimizers:
  for layer in layers:
    model = create_model(layer, optimizer)
    model.fit(x_train, y_train, epochs=15, validation_split=0.2, verbose=0)
    predict_x = model.predict(x_test)
    predictions = np.argmax(predict_x, axis=1)
    rez = accuracy_score(y_test, predictions)
    if rez > best:
      best = rez
      best_opt = optimizer
      best_layers = layer

print(best)
print(best_opt)
print(best_layers)

0.9726
adam
[(64, 'relu'), (32, 'relu')]


In [24]:
model = create_model(best_layers, best_opt)
model.fit(x_train, y_train, epochs=15)
predict_x = model.predict(x_test)
predictions = np.argmax(predict_x, axis=1)
rez = accuracy_score(y_test, predictions)
print(rez)
model.summary()

Epoch 1/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4590 - accuracy: 0.8626